# Challenge 3

In this challenge we will work on the `Orders.csv` data set in the previous [Subsetting and Descriptive Stats lab](../../lab-subsetting-and-descriptive-stats/your-code/main.ipynb). In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

# Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [12]:
import numpy as np
import pandas as pd

Next, import `Orders.csv` from the "subsetting" lab folder into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [13]:
orders = pd.read_csv("../../lab-subsetting-and-descriptive-stats/your-code/Orders.zip")
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

### Sub Problem 1: finding unique customers...

In [15]:
orders.describe()

,Unnamed: 0,InvoiceNo,year,month,day,hour,Quantity,UnitPrice,CustomerID,amount_spent
count,397924.000000,397924.000000,397924.000000,397924.000000,397924.000000,397924.000000,397924.000000,397924.000000,397924.000000,397924.000000
mean,278465.221859,560617.126645,2010.934259,7.612537,3.614555,12.728247,13.021823,3.116174,15294.315171,22.394749
std,152771.368303,13106.167695,0.247829,3.416527,1.928274,2.273535,180.420210,22.096788,1713.169877,309.055588
min,0.000000,536365.000000,2010.000000,1.000000,1.000000,6.000000,1.000000,0.000000,12346.000000,0.000000
25%,148333.750000,549234.000000,2011.000000,5.000000,2.000000,11.000000,2.000000,1.250000,13969.000000,4.680000
50%,284907.500000,561893.000000,2011.000000,8.000000,3.000000,13.000000,6.000000,1.950000,15159.000000,11.800000
75%,410079.250000,572090.000000,2011.000000,11.000000,5.000000,14.000000,12.000000,3.750000,16795.000000,19.800000
max,541908.000000,581587.000000,2011.000000,12.000000,7.000000,20.000000,80995.000000,8142.750000,18287.000000,168469.600000


Generamos otro dataframe que contenga las columnas `CustomerID`y `amount_spent`

In [60]:
amount_spent_and_customer = pd.DataFrame([orders['CustomerID'], orders['amount_spent']]).T
amount_spent_and_customer.head()

,CustomerID,amount_spent
0,17850.0,15.30
1,17850.0,20.34
2,17850.0,22.00
3,17850.0,20.34
4,17850.0,20.34


Obtenemos los valores únicos de `CustomerID`

In [61]:
customers = amount_spent_and_customer['CustomerID'].unique()
len(customers)

4339

### Continue... Sub Problem 1: aggregating the amount_spent for unique customers...

Ahora lo que vamos a hacer es agrupar los datos por `CustomerID` y después vamos a sumar los valores de 
`amount_spent`

In [62]:
amount_spent_by_customer = amount_spent_and_customer.groupby(['CustomerID']).sum().copy()
amount_spent_by_customer.head()

,amount_spent
CustomerID,
12346.0,77183.60
12347.0,4310.00
12348.0,1797.24
12349.0,1757.55
12350.0,334.40


In [105]:
amount_spent_by_customer.sort_values(by='amount_spent', ascending=False).head()

,amount_spent
CustomerID,
14646.0,280206.02
18102.0,259657.30
17450.0,194550.79
16446.0,168472.50
14911.0,143825.06


#### Sub Problem 2: Selecting customers whose aggregated `amount_spent` is in a given quantile range?

Vamos a probar por curiosidad los valores de los quantiles pedidos


In [106]:
VIP_amount = amount_spent_by_customer['amount_spent'].quantile(0.95)
VIP_amount

5840.181999999983

Obtenemos los clientes que superan el quantil 95 para obtener los clientes VIP

In [107]:
VIP_customers = amount_spent_by_customer[amount_spent_by_customer['amount_spent'] > VIP_amount]
VIP_customers.head()

,amount_spent
CustomerID,
12346.0,77183.60
12357.0,6207.67
12359.0,6372.58
12409.0,11072.67
12415.0,124914.53


### Extraemos los valores de CustomerID 

Este ha sido uno de los momentos más tensos de este lab... porque CustomerId ya no es una columna... se ha convertido en índice @!^*ç

In [140]:
VIP = list(VIP_customers.index)

### Repetimos el mismo proceso con clientes de tipo Preferred

In [97]:
Preferred_amount = amount_spent_by_customer['amount_spent'].quantile(0.75)
Preferred_amount

1661.6400000000003

In [142]:
preferred_customers = amount_spent_by_customer[(amount_spent_by_customer['amount_spent'] > Preferred_amount) & \
                                               (amount_spent_by_customer['amount_spent'] < VIP_amount)]
preferred_customers.head()

,amount_spent
CustomerID,
12347.0,4310.00
12348.0,1797.24
12349.0,1757.55
12352.0,2506.04
12356.0,2811.43


In [125]:
Preferred = list(preferred_customers.index)

#### Sub Problem 3: Labeling (or trying to labeling) selected customers as "VIP" or "Preferred"?

Todo el trabajo anterior viene definido por este paso.
La forma más *"¿fácil?"* era generar una nueva columna en orders que fuera `customers_type` y tendrían un valor u otro en función del valor que tuviese `CustomerID` porque pertenecería a uno de los dataframes creados y no al otro. El resto serían normales.  

#### *Nota para el futuro:*  
Cuando trabajamos con np.where() tenemos que tener en cuenta que sólo acepta 3 parámetros:  
* Condición
* Valor en caso de cumplir la condición
* Valor en caso de NO cumplir la condición  


Por tanto, si queremos encadenar condiciones tenemos que hacer igual que con los bloques `if`, encadernar uno dentro del otro, SIN olvidarnos del `if`





In [139]:
orders['customers_type'] = np.where(orders['CustomerID'].isin(VIP), 'VIP', \
                                   np.where(orders['CustomerID'].isin(Preferred) , 'Preferred','Normal'))

orders.tail(30)

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent,customers_type
397894,541879,581585,22726,2011,12,5,12,alarm clock bakelike green,8,2011-12-09 12:31:00,3.75,15804,United Kingdom,30.00,Preferred
397895,541880,581585,22727,2011,12,5,12,alarm clock bakelike red,4,2011-12-09 12:31:00,3.75,15804,United Kingdom,15.00,Preferred
397896,541881,581585,16016,2011,12,5,12,large chinese style scissor,10,2011-12-09 12:31:00,0.85,15804,United Kingdom,8.50,Preferred
397897,541882,581585,21916,2011,12,5,12,set 12 retro white chalk sticks,24,2011-12-09 12:31:00,0.42,15804,United Kingdom,10.08,Preferred
397898,541883,581585,84692,2011,12,5,12,box of 24 cocktail parasols,25,2011-12-09 12:31:00,0.42,15804,United Kingdom,10.50,Preferred
397899,541884,581585,84946,2011,12,5,12,antique silver t-light glass,12,2011-12-09 12:31:00,1.25,15804,United Kingdom,15.00,Preferred
397900,541885,581585,21684,2011,12,5,12,small medina stamped metal bowl,12,2011-12-09 12:31:00,0.85,15804,United Kingdom,10.20,Preferred
397901,541886,581585,22398,2011,12,5,12,magnets pack of 4 swallows,12,2011-12-09 12:31:00,0.39,15804,United Kingdom,4.68,Preferred
397902,541887,581585,23328,2011,12,5,12,set 6 school milk bottles in crate,4,2011-12-09 12:31:00,3.75,15804,United Kingdom,15.00,Preferred
397903,541888,581585,23145,2011,12,5,12,zinc t-light holder star large,12,2011-12-09 12:31:00,0.95,15804,United Kingdom,11.40,Preferred


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

# Q2: How to identify which country has the most VIP Customers?

# Q3: How to identify which country has the most VIP+Preferred Customers combined?

Provide your solution for Q2 below:

### Q2: How to identify which country has the most VIP Customers?

In [151]:
vip_df = orders[orders['customers_type'] == 'VIP']
vip_countries = list(vip_df['Country'].unique())
vip_countries

['France',
 'United Kingdom',
 'Australia',
 'Norway',
 'EIRE',
 'Spain',
 'Germany',
 'Portugal',
 'Japan',
 'Sweden',
 'Netherlands',
 'Cyprus',
 'Switzerland',
 'Singapore',
 'Belgium',
 'Finland',
 'Channel Islands',
 'Denmark']

Primero intentamos sacar los clientes por país con el ejemplo de Francia para, a partir  de él, obtener una función

In [162]:
france_vip_costumers = list(vip_df[vip_df['Country'] == 'France']['CustomerID'].unique())
france_vip_costumers

[12583, 12682, 12567, 12681, 12683, 12731, 12678, 12637, 12536]

La función siguiente permite obtener un diccionario cuyas claves son los países de clientes VIP y sus valores la lista de clientes.  

#### Parámetros de entrada:  
* Dataframe
* Columna que queremos filtrar
* Lista de valores condicionantes
* Columna cuyos valores queremos obtener

#### Resultado:  
* Diccionario cuyas claves son los elementos de la lista `condition` y sus valores los correspondientes a esa condición en la columna `col`

In [174]:
def get_values_coditioned_by_a_value(df, col, condition, col2):
    result = {}
    for item in condition:
        result[item] = list(df[df[col] == item][col2].unique())
    return result


In [173]:
result = get_values_coditioned_by_a_value(vip_df, 'Country', vip_countries, 'CustomerID')
result

{'France': [12583, 12682, 12567, 12681, 12683, 12731, 12678, 12637, 12536],
 'United Kingdom': [15311,
  16029,
  17511,
  13408,
  13767,
  15513,
  13694,
  14849,
  16210,
  12748,
  17841,
  13093,
  12921,
  13777,
  18229,
  14606,
  13576,
  13090,
  15694,
  17017,
  15601,
  13418,
  14060,
  17381,
  17581,
  15061,
  15640,
  14031,
  12971,
  13798,
  17396,
  14680,
  16013,
  17949,
  15769,
  13081,
  17243,
  15465,
  13089,
  16033,
  18055,
  18109,
  16839,
  16814,
  16353,
  14527,
  15023,
  16422,
  15502,
  17677,
  17428,
  15039,
  15078,
  14667,
  15194,
  17450,
  17735,
  15838,
  14733,
  13488,
  17675,
  18102,
  13078,
  16779,
  14796,
  13199,
  17706,
  16525,
  16558,
  15498,
  14051,
  16713,
  13113,
  15005,
  14866,
  17340,
  18092,
  15358,
  13319,
  13854,
  17857,
  15856,
  13102,
  13969,
  16656,
  14952,
  12989,
  17865,
  16873,
  14062,
  16923,
  13668,
  15044,
  14505,
  13225,
  13209,
  17338,
  15159,
  13324,
  14961,
  1405